In [22]:
import pandas as pd
file = pd.read_csv('True-news.csv')
file[:5]

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [23]:
file['subject'].nunique()

2

In [24]:
file['subject'].value_counts()

politicsNews    11272
worldnews       10145
Name: subject, dtype: int64

In [28]:
dataset = file[['text', 'subject']]
dataset[:5]

,text,subject
0,WASHINGTON (Reuters) - The head of a conservat...,politicsNews
1,WASHINGTON (Reuters) - Transgender people will...,politicsNews
2,WASHINGTON (Reuters) - The special counsel inv...,politicsNews
3,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews


In [30]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(dataset['subject'])
encoder.classes_

array(['politicsNews', 'worldnews'], dtype=object)

In [31]:
dataset['label'] = encoder.transform(dataset['subject'])
dataset[:5]

C:\Users\AZK\AppData\Local\Temp\ipykernel_16204\1810508170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['label'] = encoder.transform(dataset['subject'])


,text,subject,label
0,WASHINGTON (Reuters) - The head of a conservat...,politicsNews,0
1,WASHINGTON (Reuters) - Transgender people will...,politicsNews,0
2,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,0
3,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,0
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,0


In [32]:
label_names = encoder.classes_
label_names

array(['politicsNews', 'worldnews'], dtype=object)

In [34]:
split = int(0.8 * len(dataset))
train_dataset = dataset[:split]
test_dataset = dataset[split:]

In [35]:
len(train_dataset), len(test_dataset)

(17133, 4284)

In [53]:
x_train = train_dataset['text'].to_numpy()
y_train = train_dataset['label'].to_numpy()
x_test = test_dataset['text'].to_numpy()
y_test = test_dataset['label'].to_numpy()
len(x_train), len(y_train), len(x_test), len(y_test)

(17133, 17133, 4284, 4284)

In [64]:
from keras.layers import TextVectorization, Embedding
text_vectorization = TextVectorization(max_tokens=20000, output_mode='int', output_sequence_length=128)
text_vectorization.adapt(x_train)
embedding = Embedding(input_dim=20000, output_dim=128, input_length=128)

In [57]:
import tensorflow as tf
from keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
text_vectorization = text_vectorization(inputs)
embedding = embedding(text_vectorization)
x = layers.LSTM(16, activation='relu')(embedding)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.models.Model(inputs, outputs)

callbacks = [tf.keras.callbacks.ModelCheckpoint('news models/model_1_lstm', save_best_only=True)]


model_1.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=['accuracy']
)

history_model_1 = model_1.fit(
    x=x_train,
    y=y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    callbacks=callbacks
)

Epoch 1/10
536/536 [==============================] - 21s 37ms/step - loss: 11.3741 - accuracy: 0.6682 - val_loss: 0.8925 - val_accuracy: 0.2178
Epoch 2/10
536/536 [==============================] - 18s 33ms/step - loss: 135561.9375 - accuracy: 0.8643 - val_loss: 467.7920 - val_accuracy: 0.9435
Epoch 3/10
536/536 [==============================] - 18s 33ms/step - loss: 47.9874 - accuracy: 0.8860 - val_loss: 47.6820 - val_accuracy: 0.9468
Epoch 4/10
536/536 [==============================] - 18s 33ms/step - loss: 0.5395 - accuracy: 0.9136 - val_loss: 117.7237 - val_accuracy: 0.9456
Epoch 5/10
536/536 [==============================] - 18s 33ms/step - loss: 0.5279 - accuracy: 0.9231 - val_loss: 107.0030 - val_accuracy: 0.9526
Epoch 6/10
536/536 [==============================] - 18s 33ms/step - loss: 0.2907 - accuracy: 0.9320 - val_loss: 144.9001 - val_accuracy: 0.9512
Epoch 7/10
536/536 [==============================] - 18s 33ms/step - loss: 600.7690 - accuracy: 0.9399 - val_loss: 82.7

In [59]:
model_1.evaluate(x_test, y_test)

134/134 [==============================] - 1s 9ms/step - loss: 15.5498 - accuracy: 0.9185


[15.549821853637695, 0.9185341000556946]

In [62]:
model_1 = tf.keras.models.load_model('news models/model_1_lstm')
model_1.evaluate(x_test, y_test)

134/134 [==============================] - 1s 9ms/step - loss: 0.8925 - accuracy: 0.2178


[0.8925114274024963, 0.21778711676597595]

In [65]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
text_vectorization = text_vectorization(inputs)
embedding = embedding(text_vectorization)
x = layers.LSTM(16, return_sequences=True, activation='relu', recurrent_dropout=0.25)(embedding)
x = layers.LSTM(16, activation='relu', recurrent_dropout=0.25)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.models.Model(inputs, outputs)

callbacks = [tf.keras.callbacks.ModelCheckpoint('news models/model_2_lstm_dropout', save_best_only=True)]

model_2.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history_model_2 = model_2.fit(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    callbacks=callbacks
)

Epoch 1/10
536/536 [==============================] - ETA: 0s - loss: 145.5624 - accuracy: 0.6845

INFO:tensorflow:Assets written to: news models\model_2_lstm_dropout\assets


INFO:tensorflow:Assets written to: news models\model_2_lstm_dropout\assets


536/536 [==============================] - 49s 86ms/step - loss: 145.5624 - accuracy: 0.6845 - val_loss: 65.3809 - val_accuracy: 0.0219
Epoch 2/10
536/536 [==============================] - ETA: 0s - loss: 2.5574 - accuracy: 0.6922

INFO:tensorflow:Assets written to: news models\model_2_lstm_dropout\assets


...

INFO:tensorflow:Assets written to: news models\model_2_lstm_dropout\assets


536/536 [==============================] - 45s 84ms/step - loss: 0.3848 - accuracy: 0.8161 - val_loss: 1.0116 - val_accuracy: 0.8109
Epoch 10/10
536/536 [==============================] - ETA: 0s - loss: 0.3596 - accuracy: 0.8359

INFO:tensorflow:Assets written to: news models\model_2_lstm_dropout\assets


INFO:tensorflow:Assets written to: news models\model_2_lstm_dropout\assets


536/536 [==============================] - 48s 90ms/step - loss: 0.3596 - accuracy: 0.8359 - val_loss: 0.9283 - val_accuracy: 0.8459


In [66]:
model_2.evaluate(x_test, y_test)

134/134 [==============================] - 2s 18ms/step - loss: 0.9283 - accuracy: 0.8459


[0.9282684326171875, 0.8459383845329285]

In [ ]:
model_2_loaded = tf.keras.models.load_model('news models/model_2_lstm_dropout')
model_2_loaded.evaluate(x_test, y_test)